In [1]:
#importing libraries

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imdb-movie-reviews-dataset/test_data (1).csv
/kaggle/input/imdb-movie-reviews-dataset/train_data (1).csv


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [3]:
#reading the data

test_csv = pd.read_csv('/kaggle/input/imdb-movie-reviews-dataset/test_data (1).csv') 
train_csv = pd.read_csv('/kaggle/input/imdb-movie-reviews-dataset/train_data (1).csv') 

In [4]:
#stopword removal and lemmatization

stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [5]:
# Preprocessing
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
train_csv.head()

,0,1
0,"This film is absolutely awful, but nevertheles...",0
1,Well since seeing part's 1 through 3 I can hon...,0
2,I got to see this film at a preview and was da...,1
3,This adaptation positively butchers a classic ...,0
4,Råzone is an awful movie! It is so simple. It ...,0


In [7]:
train_X_non = train_csv['0']   # '0' refers to the review text
train_y = train_csv['1']   # '1' corresponds to Label (1 - positive and 0 - negative)
test_X_non = test_csv['0']
test_y = test_csv['1']

train_X=[]
test_X=[]

# Text Pre Processing

In [8]:
#text pre processing

for i in range(0, len(train_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', train_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)

In [9]:
#text pre processing

for i in range(0, len(test_X_non)):
    review = re.sub('[^a-zA-Z]', ' ', test_X_non[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)

In [10]:
train_X[10]

'eddie murphy delirious far funniest thing ever see life compare movie garuntee decide delirious funniest movie ever movie hr min throughout time barely moment laughing laugh hour replaying punch line head eddie murphy given many funny performance career hr trading place beverly hill cop raw coming america nutty professor shrek etc far hilarious moment seen movie many time funnier every time never loses edge day forward every great stand performance emulated delirious two thumb'

In [11]:
test_X[10]

'surprise maker hopeless movie find uk distributor release free dvd sunday newspaper distributor could clearly see film maker sunday newspaper one movie going recoup cost br br since thriller riddle would helped picked lead actor could enunciate properly rather mumbling vinnie jones appears pronounce riddle riell would helped dialogue swamped noisy location scene flooded distracting inappropriate music plot ludicrous lost charles dickens story supposedly help hero solve series modern murder would copy herge adventure tintin since link dickens jones non existent tenuous ridiculous premise would investigative journalist lay hand previously undiscovered dickens manuscript would take several day read flashback dickens continue played throughout movie connection mean found new dickens manuscript go somewhere quiet read film end one surprise revelation become mandatory since sixth sense case much surprise insult intelligence film suddenly going turn supernatural twelfth hour revealing vinnie

In [12]:
#tf idf

tf_idf = TfidfVectorizer()

In [13]:
#applying tf idf to training data

X_train_tf = tf_idf.fit_transform(train_X)

In [14]:
#applying tf idf to training data

X_train_tf = tf_idf.transform(train_X)

In [15]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 25000, n_features: 65498


In [16]:
#transforming test data into tf-idf matrix

X_test_tf = tf_idf.transform(test_X)

In [17]:
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

n_samples: 25000, n_features: 65498


# Model Creation

In [18]:
#naive bayes classifier

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

MultinomialNB()

In [19]:
#predicted y

y_pred = naive_bayes_classifier.predict(X_test_tf)

In [20]:
print(metrics.classification_report(test_y, y_pred,
                                            target_names=['Positive', 'Negative']))

              precision    recall  f1-score   support

    Positive       0.80      0.88      0.84     12500
    Negative       0.87      0.78      0.82     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000



In [21]:
print("Confusion matrix:")
print(metrics.confusion_matrix(test_y, y_pred))

Confusion matrix:
[[10990  1510]
 [ 2693  9807]]


# Doing a Test Prediction

In [22]:

#doing a test prediction

test = ["This is unlike any kind of adventure movie my eyes have ever seen in such a long time, the characters, the musical score for every scene, the story, the beauty of the landscapes of Pandora, the rich variety and uniqueness of the flora and fauna of Pandora, the ways and cultures and language of the natives of Pandora, everything about this movie I am beyond impressed and truly captivated by. Sam Worthington is by far my favorite actor in this movie along with his character Jake Sulley, just as he was a very inspiring actor in The Shack Sam Worthington once again makes an unbelievable mark in one of the greatest and most captivating movies you'll ever see. "]

In [23]:
review = re.sub('[^a-zA-Z]', ' ', test[0])
review = review.lower()
review = review.split()
review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
test_processed =[ ' '.join(review)]


In [24]:
test_processed

['unlike kind adventure movie eye ever seen long time character musical score every scene story beauty landscape pandora rich variety uniqueness flora fauna pandora way culture language native pandora everything movie beyond impressed truly captivated sam worthington far favorite actor movie along character jake sulley inspiring actor shack sam worthington make unbelievable mark one greatest captivating movie ever see']

In [25]:
test_input = tf_idf.transform(test_processed)

In [26]:
test_input.shape

(1, 65498)

In [27]:

#0= bad review
#1= good review

res=naive_bayes_classifier.predict(test_input)[0]

if res==1:
    print("Good Review")
    
elif res==0:
    print("Bad Review")

Good Review
